<a href="https://colab.research.google.com/github/gitmitsu13/cursor-pj/blob/main/main.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
pip install google-api-python-client google-auth-httplib2 google-auth-oauthlib

In [13]:
# Colabの場合
from google.colab import drive
import os

# Googleドライブをマウント
drive.mount('/content/drive')

# credentials.jsonへのパスを指定（例：マイドライブの「MyProject」フォルダにある場合）
# 必要に応じてパスを修正してください
credentials_path = '"C:\Users\jobmi\Documents\GitHub\ToDoList_to_Spreadsheet_create_table\credentials.json.json"'

def authenticate_google_api():
    """Google APIに認証し、認証情報を返します。"""
    creds = None
    # token.jsonへのパスも必要に応じて修正
    token_path = 'token.json' # 同じディレクトリに保存する場合

    if os.path.exists(token_path):
        creds = Credentials.from_authorized_user_file(token_path, SCOPES)
    if not creds or not creds.valid:
        if creds and creds.expired and creds.refresh_token:
            creds.refresh(Request())
        else:
            # ファイルパスを引数に渡す
            flow = InstalledAppFlow.from_client_secrets_file(
                credentials_path, SCOPES)
            creds = flow.run_local_server(port=0)
        with open(token_path, 'w') as token:
            token.write(creds.to_json())
    return creds

# main関数や他の部分は変更なし
# ... (残りのコードはそのまま)

def get_tasks(creds):
    """Google ToDoリストからすべてのタスクを取得します。"""
    tasks_service = build('tasks', 'v1', credentials=creds)
    all_tasks_data = []

    try:
        task_lists_result = tasks_service.tasklists().list().execute()
        task_lists = task_lists_result.get('items', [])

        if not task_lists:
            print("タスクリストが見つかりませんでした。")
            return []

        print("取得したタスクリスト:")
        for task_list in task_lists:
            print(f"- {task_list['title']} (ID: {task_list['id']})")

            tasks_result = tasks_service.tasks().list(tasklist=task_list['id']).execute()
            tasks = tasks_result.get('items', [])

            if not tasks:
                print(f"  '{task_list['title']}' にタスクがありませんでした。")
                continue

            for task in tasks:
                title = task.get('title', 'N/A')
                due = task.get('due', 'N/A')
                status = task.get('status', 'N/A')
                notes = task.get('notes', 'N/A')

                all_tasks_data.append({
                    'タスクリスト': task_list['title'],
                    'タスク名': title,
                    '期限': due,
                    'ステータス': status,
                    'メモ': notes
                })
    except HttpError as err:
        print(f"APIエラーが発生しました: {err}")
    return all_tasks_data

def write_to_spreadsheet(creds, data):
    """取得したタスクデータをGoogleスプレッドシートに書き込みます。"""
    if not data:
        print("書き込むデータがありません。")
        return

    sheets_service = build('sheets', 'v4', credentials=creds)
    df = pd.DataFrame(data)

    header = df.columns.tolist()
    values = df.values.tolist()

    body = {
        'values': [header] + values
    }

    try:
        result = sheets_service.spreadsheets().values().update(
            spreadsheetId=SPREADSHEET_ID, range=RANGE_NAME,
            valueInputOption='RAW', body=body).execute()
        print(f"スプレッドシートに {result.get('updatedCells')} セル書き込まれました。")
    except HttpError as err:
        print(f"スプレッドシートへの書き込み中にエラーが発生しました: {err}")

def main():
    creds = authenticate_google_api()
    if creds:
        tasks_data = get_tasks(creds)
        if tasks_data:
            write_to_spreadsheet(creds, tasks_data)
        else:
            print("タスクデータを取得できませんでした。")
    else:
        print("Google APIの認証に失敗しました。")


if __name__ == '__main__':
    main()

SyntaxError: (unicode error) 'unicodeescape' codec can't decode bytes in position 3-4: truncated \UXXXXXXXX escape (ipython-input-13-308894994.py, line 10)